In [1]:
%load_ext autoreload
%autoreload 2
import h5py
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
from scanfields import Scanfield, Scanfield_channel, Scanfield_total

channel_list = [
    'L1-040','L2-050','L1-060','L3-068','L2-068','L4-078','L1-078','L3-089','L2-089','L4-100','L3-119','L4-140',
    'M1-100','M2-119','M1-140','M2-166','M1-195',
    'H1-195','H2-235','H1-280','H2-337','H3-402'
]


In [2]:
class Field:
    def __init__(self, field, spin):
        if all(isinstance(x, float) for x in field):
            self.field = field + 1j*np.zeros(len(field))
        else:
            self.field = field
        self.spin = spin

class Signalfields:
    def __init__(self, *fields: Field):
        self.fields = sorted(fields, key=lambda field: field.spin)
        self.spins = np.array([field.spin for field in self.fields])


def couple_fields(scan_fields: Scanfield, signal_fields: Signalfields, spin_out: int):
    results = []
    for i in range(len(signal_fields.spins)):
        n = spin_out - signal_fields.spins[i]
        print(f"n-n': {n}, n: {signal_fields.spins[i]}")
        results.append(scan_fields.get_xlink(n) * signal_fields.fields[i].field)
    return np.array(results)

In [3]:
# Load single detector map
base_path = "../crosslinks_2407"
sf1 = Scanfield(base_path+"/boresight/nside128_boresight.h5")
maps = hp.read_map("./cmb_0000_nside_128_date_230813_seed_33.fits", field=(0,1,2)) * 1e6
#sf1 = Scanfield(base_path+"/L1-040/nside128_000_000_003_QA_040_T.h5")
#sf2 = Scanfield(base_path+"/L1-040/nside128_000_004_005_UA_040_T.h5")

In [4]:
g_a = 0.0
g_b = 0
delta_g = g_a - g_b
P = maps[1] + 1j*maps[2]

signal_field = Signalfields(
    Field(delta_g * maps[0], spin=0),
    Field(0.5 * (2.0+g_a+g_b) * P, spin=2),
    Field(0.5 * (2.0+g_a+g_b) * P.conj(), spin=-2),
)


In [5]:
s0 = couple_fields(sf1, signal_field, 0).sum(0)
s2 = couple_fields(sf1, signal_field, 2).sum(0)
sm2 = couple_fields(sf1, signal_field, -2).sum(0)

n-n': 2, n: -2
n-n': 0, n: 0
n-n': -2, n: 2
n-n': 4, n: -2
n-n': 2, n: 0
n-n': 0, n: 2
n-n': 0, n: -2
n-n': -2, n: 0
n-n': -4, n: 2


In [18]:
def get_covmat(sf):
    mat = np.array([
        [sf.get_xlink(0) ,  sf.get_xlink(-2)   , sf.get_xlink(2)],
        [sf.get_xlink(2) ,  sf.get_xlink(0)/4.0, sf.get_xlink(4)],
        [sf.get_xlink(-2), sf.get_xlink(-4)    , sf.get_xlink(0)/4.0]
        ])
    return mat

get_covmat(sf1)[:,:,0]

array([[ 1.        +0.j        , -0.07950018+0.16274038j,
        -0.07950018-0.16274038j],
       [-0.07950018-0.16274038j,  0.25      +0.j        ,
         0.0484991 +0.02470542j],
       [-0.07950018+0.16274038j,  0.0484991 -0.02470542j,
         0.25      +0.j        ]], dtype=complex64)

In [12]:
np.array([
    [sf1.get_xlink(1), sf1.get_xlink(1)],
    [sf1.get_xlink(1), sf1.get_xlink(1)],
    ]).shape

(2, 2, 196608)